In [1]:
import pandas as pd
import polars as pl

df1 = pl.DataFrame(
    [
        {
            "Date": "2023-02-01 10:00:01",
            "Vendor Name": "Golden Oil LLC",
            "Amount USD": 49.99,
            "Category": "Personal",
        }
    ]
)

df2 = pl.DataFrame(
    [
        {
            "transaction_date": "2023-03-01 10:00:01",
            "vendor_name": "Get Cars Inc.",
            "amount_usd": 2000,
            "category": "Transportation",
        }
    ]
)
df3 = pl.DataFrame(
    [
        {
            "DATE": "2023-04-01 10:00:01",
            "VENDOR_NAME": "Maintainers Exc.",
            "USD": 5249.0,
            "CAT": "Personal",
            "foo": "Personal",
        }
    ]
)

In [2]:
df1

Date,Vendor Name,Amount USD,Category
str,str,f64,str
"""2023-02-01 10:00:01""","""Golden Oil LLC""",49.99,"""Personal"""


In [3]:
df2

transaction_date,vendor_name,amount_usd,category
str,str,i64,str
"""2023-03-01 10:00:01""","""Get Cars Inc.""",2000,"""Transportation"""


In [4]:
df3

DATE,VENDOR_NAME,USD,CAT,foo
str,str,f64,str,str
"""2023-04-01 10:00:01""","""Maintainers Exc.""",5249.0,"""Personal""","""Personal"""


In [5]:
from dapter import BaseMapper, accepts, accepts_anycases


def convert_to_eur(col: pd.Series) -> pd.Series:
    return col * 0.92


class TransactionMapper(BaseMapper):
    transaction_date = accepts("transaction_date", "Date", "DATE")
    vendor_name = accepts_anycases()
    amount_eur = (accepts("Amount USD", "amount_usd", "USD"), convert_to_eur)
    category = accepts("Category", "category", "CAT")


mapper = TransactionMapper()

dfs = mapper.apply(df1, df2, df3)
df = pl.concat(dfs)
df

transaction_date,vendor_name,amount_eur,category
str,str,f64,str
"""2023-02-01 10:00:01""","""Golden Oil LLC""",45.9908,"""Personal"""
"""2023-03-01 10:00:01""","""Get Cars Inc.""",1840.0,"""Transportation"""
"""2023-04-01 10:00:01""","""Maintainers Exc.""",4829.08,"""Personal"""
